# Moscow housing prediction [Short]

Written by<br> 

Hauk Aleksander Olaussen - **Student ID: *504903*** <br>

Charbel Badr - **Student ID: *557389***<br>

Noran Baskaran - **Student ID: *504877*** <br>

Kaggle competition name: **Moscow housing**

Kaggle team name: **Team 69**

## Introduction

This short notebook will contain the code and comments for you to reproduce our best performing model on the public testset. It will make use of the `Preprocessor.py` and `TestModel.py` classes, just as in **long.ipynb**. We will not do any exploration or engineering in this notebook other than run the functions actually doing this job in `Preprocessor.py`. We hope that this will contribute to a readable model with as little "code gore" as possible. 

In [114]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
%autoreload
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from Preprocessor import Preprocessor
from TestModel import TestModel
import lightgbm as lgbm
import pandas as pd
import numpy as np
import xgboost

preprocessor = Preprocessor()

## Extracting the data

As the data is contained within *.csv* files, we need to read this and put it into a `pandas` DataFrame. This will make it easier to perform the preprocessing later. This is done in the same way as you saw in the long notebook.

In [116]:
merged_final = pd.concat([preprocessor.merged.copy(), preprocessor.merged_test.copy()], ignore_index=True)
print(f"The dataframe now has {len(merged_final)} entries")

The dataframe now has 33222 entries


Lets run the preprocessing functions

In [117]:
general_removed = preprocessor.general_removal(merged_final.copy()) # Removes "layout", "ceiling", "balconies", "loggias", "elevator_without", "street", "address"
data = preprocessor.remove_NaNs(general_removed.copy())             # Fills in NaN valiues in different ways depending on the entry

# Fixing the lat-lon outliers as decsribed in long
is_outlier = (data["longitude"] > 55) | (data["latitude"] < 54)
outliers = data.copy()[is_outlier]
data = preprocessor.fix_latlon_outliers(data.copy(), outliers)
data.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,bathrooms_shared,bathrooms_private,...,longitude,district,constructed,material,stories,elevator_passenger,elevator_service,parking,garbage_chute,heating
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,0.0,2.0,...,37.478055,11.0,2021.0,3.0,9.0,1.0,1.0,1.0,-1.0,-1.0
1,1,-1.0,10500000.0,88.0,14.2,48.0,18.0,3.0,2.0,0.0,...,37.666647,11.0,2010.0,3.0,25.0,1.0,1.0,1.0,-1.0,0.0
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,0.0,2.0,...,37.515335,11.0,2021.0,3.0,15.0,1.0,1.0,1.0,-1.0,-1.0
3,3,-1.0,10500000.0,88.0,14.0,48.0,18.0,3.0,0.0,2.0,...,37.666647,11.0,2010.0,3.0,25.0,1.0,1.0,1.0,-1.0,0.0
4,4,-1.0,13900000.0,78.0,17.0,35.0,7.0,2.0,1.0,0.0,...,37.451438,11.0,2017.0,2.0,15.0,1.0,1.0,1.0,0.0,0.0


## Adding features

Now lets start adding the features we want for the model. We start with the distance features

In [118]:
# Adding distance features
distance_df = preprocessor.combine_latlon_subway(data.copy(), read_from_file=True)  # Subway
distance_df = preprocessor.closest_hospital(distance_df.copy())                     # Hospital
distance_df = preprocessor.closest_park(distance_df.copy())                         # Parks 
distance_df = preprocessor.distance_luxury_village(distance_df.copy())              # Luxury village
distance_df = preprocessor.closest_uni(distance_df.copy())                          # University
distance_df = preprocessor.inside_golden_mile(distance_df.copy())                   # Golden mile, boolean
distance_df = preprocessor.distance_from_golden_mile(distance_df.copy())            # Golden mile, distance
distance_df = preprocessor.inside_khamovniki(distance_df.copy())                    # Khamovniki, boolean
distance_df = preprocessor.distance_from_khamovniki_center(distance_df.copy())      # Khamovniki, distance
distance_df = preprocessor.combine_latlon(distance_df.copy())                       # Red square
distance_df = preprocessor.closest_powerplant(distance_df.copy())                   # Power plant
print("Distance features added")

Distance features added


We continue by adding combined features.

In [119]:
# Creating the new features
featured = preprocessor.combine_windows(distance_df.copy(), boolean=True) # has_windows feature
featured = preprocessor.redo_new(featured.copy())                         # Fixing new feature before combining
featured = preprocessor.combine_new_constructed_distance(featured.copy()) # scaled_constructed feature
featured = preprocessor.floor_fraction(featured.copy())                   # floor_fraction feature
featured = preprocessor.combine_floor_stories(featured.copy())            # floor_stories feature
featured = preprocessor.area_distance(featured.copy())                    # area_distance feature
featured = preprocessor.combine_baths(featured.copy())                    # bathroom_amount feature
featured.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,bathrooms_shared,bathrooms_private,...,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance,bathroom_amount
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,0.0,2.0,...,0,21.990803,25.022080,10.364623,True,2021.000000,0.222222,18.0,0.171074,2.0
1,1,-1.0,10500000.0,88.0,14.2,48.0,18.0,3.0,2.0,0.0,...,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,0.0,2.0,...,0,8.719216,12.060134,6.032235,True,2021.000000,0.800000,180.0,0.119716,2.0
3,3,-1.0,10500000.0,88.0,14.0,48.0,18.0,3.0,0.0,2.0,...,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
4,4,-1.0,13900000.0,78.0,17.0,35.0,7.0,2.0,1.0,0.0,...,0,17.732445,21.041782,11.287607,True,2013.007917,0.466667,105.0,0.127619,1.0


## Removing features

Lets now remove features we do not indend to keep.

In [120]:
to_be_removed = ["rooms", "seller", "floor", "garbage_chute", "heating", "phones","area_kitchen", "area_living", "id", "constructed", "bathrooms_private", "bathrooms_shared", "new", "windows_street", "stories", "windows_court", "elevator_passenger", "elevator_service"]
removed = preprocessor.remove_labels(featured.copy(), to_be_removed)
removed.head()

,price,area_total,condition,building_id,latitude,longitude,district,material,parking,closest_subway_distance,...,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance,bathroom_amount
0,7139520.0,59.2,-1.0,4076,55.544046,37.478055,11.0,3.0,1.0,1.911344,...,0,21.990803,25.022080,10.364623,True,2021.000000,0.222222,18.0,0.171074,2.0
1,10500000.0,88.0,3.0,1893,55.861282,37.666647,11.0,3.0,1.0,0.913793,...,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
2,9019650.0,78.5,-1.0,5176,55.663299,37.515335,11.0,3.0,1.0,1.643014,...,0,8.719216,12.060134,6.032235,True,2021.000000,0.800000,180.0,0.119716,2.0
3,10500000.0,88.0,2.0,1893,55.861282,37.666647,11.0,3.0,1.0,0.913793,...,0,15.264384,12.267029,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
4,13900000.0,78.0,3.0,6604,55.590785,37.451438,11.0,2.0,1.0,1.220328,...,0,17.732445,21.041782,11.287607,True,2013.007917,0.466667,105.0,0.127619,1.0


## Logifying skewed data

We logify some of the data, as they are quite skewed originally.

In [121]:
finished = preprocessor.logify(removed.copy(), "area_total")
finished = preprocessor.logify(finished.copy(), "distance_center")
finished = preprocessor.logify(finished.copy(), "price")
finished.head()

,price,area_total,condition,building_id,latitude,longitude,district,material,parking,closest_subway_distance,...,inside_khamovniki,distance_khamovniki_center,distance_center,closest_powerplant,has_windows,scaled_constructed,floor_fraction,floor_stories,area_distance,bathroom_amount
0,15.781156,4.097672,-1.0,4076,55.544046,37.478055,11.0,3.0,1.0,1.911344,...,0,21.990803,3.258945,10.364623,True,2021.000000,0.222222,18.0,0.171074,2.0
1,16.166886,4.488636,3.0,1893,55.861282,37.666647,11.0,3.0,1.0,0.913793,...,0,15.264384,2.585282,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
2,16.014916,4.375757,-1.0,5176,55.663299,37.515335,11.0,3.0,1.0,1.643014,...,0,8.719216,2.569564,6.032235,True,2021.000000,0.800000,180.0,0.119716,2.0
3,16.166886,4.488636,2.0,1893,55.861282,37.666647,11.0,3.0,1.0,0.913793,...,0,15.264384,2.585282,7.957621,True,1999.059871,0.720000,450.0,0.106986,2.0
4,16.447399,4.369448,3.0,6604,55.590785,37.451438,11.0,2.0,1.0,1.220328,...,0,17.732445,3.092940,11.287607,True,2013.007917,0.466667,105.0,0.127619,1.0


## Preparing for the model

We will now split the finished DataFrame back into training and test data. The training data contains prices, which we have put into a variable called *labels* above. The test data does not have prices - as this is the task at hand. We will use a combination of *XGBoost* and *LGBM* for our prediction. The configurations have been found using Microsofts autoML library *flaml*.

#### XGBoost

In [158]:
train_data = finished[:23285].copy()                 # Data used for fitting
test_data = finished[23285:].copy()                  # Actual data we want to predict

train_data = preprocessor.fix_categorical(train_data.copy()) # Changes the type of categorical data to "category" instead of float
test_data = preprocessor.fix_categorical(test_data.copy()) # Changes the type of categorical data to "category" instead of float

labels = train_data["price"]                         # Learning labels

train_data.drop("price", 1, inplace=True)            # Removing labels from training set
test_data.drop("price", 1, inplace=True)             # Removing labels from training set, even though they are all NaN'
print("Number of features:", train_data.shape[1])

Number of features: 25


In [159]:
xgboost_model = TestModel(train_data, labels)
xgb_config = {'n_estimators': 2633, 'max_leaves': 46, 'min_child_weight': 44.96191321223358, 'learning_rate': 0.031513272281011175, 'subsample': 0.9802999582217468, 'colsample_bylevel': 0.3320103632430048, 'colsample_bytree': 0.39735916748894895, 'reg_alpha': 0.003927651042764543, 'reg_lambda': 1.7739677389468846}
xgb_model = xgboost_model.xgboost_model(xgb_config)
xgb_pipe = Pipeline([("scaler", StandardScaler()), ("xbg", xgb_model)])

In [160]:
print("Fitting XGBoost model...")
xgb_fitted = xgb_pipe.fit(train_data, labels)
print("XGBoost fitting complete!")

Fitting XGBoost model...
XGBoost fitting complete!


In [162]:
xgb_prediction = xgb_fitted.predict(test_data)
xgb_predicted_prices = np.expm1(xgb_prediction)
print("XGBoost prediction finished!")
xgb_predicted_prices

XGBoost prediction finished!


array([28888416. ,  9853941. ,  6029955.5, ...,  9392801. ,  9688152. ,
        6523811.5], dtype=float32)

#### LGBM

In [165]:
lgbm_model = TestModel(train_data, labels)
lgbm_config = {'n_estimators': 8960, 'num_leaves': 13, 'min_child_samples': 7, 'learning_rate': 0.014186627557379268, 'log_max_bin': 10, 'colsample_bytree': 0.6863629738189831, 'reg_alpha': 0.0016409881781172429, 'reg_lambda': 0.004709029793085598}
lgbm_model = lgbm_model.lgbm_model(lgbm_config)
lgbm_pipe = Pipeline([("scaler", StandardScaler()), ("lgbm", lgbm_model)])

In [166]:
print("Fitting LGBM model...")
lgbm_fitted = lgbm_pipe.fit(train_data, labels)
print("LGBM fitting complete!")

Fitting LGBM model...
LGBM fitting complete!


In [167]:
lgbm_prediction = lgbm_fitted.predict(test_data)
lgbm_predicted_prices = np.expm1(lgbm_prediction)
print("LGBM  prediction finished!")
lgbm_predicted_prices

LGBM  prediction finished!


array([27256528.93875849, 10540447.98571461,  6274542.33445403, ...,
        9534272.34154317,  9658755.40006378,  5851434.63286338])

#### Combining predictions

We will now combined the results from these two by taking the average value they predicted

In [169]:
combined_prediction = (xgb_predicted_prices + lgbm_predicted_prices) / 2
xgboost_model.save_predictions(list(combined_prediction))

,id,price_prediction
0,23285,2.807247e+07
1,23286,1.019719e+07
2,23287,6.152249e+06
3,23288,8.156949e+06
4,23289,5.161018e+06
...,...,...
9932,33217,2.636738e+07
9933,33218,1.707424e+07
9934,33219,9.463537e+06
9935,33220,9.673454e+06


In [189]:
train_data = finished[:23285].copy()
test_data = finished[23285:].copy()
test_data.drop("price", 1, inplace=True)

# Splitting training data into training and validation, removed the price for each of them afterwards
x_train, x_test, y_train, y_test = train_test_split(train_data, labels, stratify=train_data.price.round(), test_size=0.3)
x_train.drop("price", 1, inplace=True)
x_test.drop("price", 1, inplace=True)
x_train = preprocessor.fix_categorical(x_train.copy()) # Changes the type of categorical data to "category" instead of float
x_test = preprocessor.fix_categorical(x_test.copy()) # Changes the type of categorical data to "category" instead of float
print(f"Number of features:", x_train.shape[1])

Number of features: 25


In [ ]:
xgboost_model = TestModel(train_data, labels)
xgb_config = {'n_estimators': 2633, 'max_leaves': 46, 'min_child_weight': 44.96191321223358, 'learning_rate': 0.031513272281011175, 'subsample': 0.9802999582217468, 'colsample_bylevel': 0.3320103632430048, 'colsample_bytree': 0.39735916748894895, 'reg_alpha': 0.003927651042764543, 'reg_lambda': 1.7739677389468846}
xgb_model = xgboost_model.xgboost_model(xgb_config)
xgb_pipe = Pipeline([("scaler", StandardScaler()), ("xbg", xgb_model)])
xgb_fitted = xgb_pipe.fit(x_train, y_train)
xgb_prediction = xgb_fitted.predict(x_test)
xgb_predicted_prices = np.expm1(xgb_prediction)
xgb_predicted_prices

lgbm_model = TestModel(train_data, labels)
lgbm_config = {'n_estimators': 8960, 'num_leaves': 13, 'min_child_samples': 7, 'learning_rate': 0.014186627557379268, 'log_max_bin': 10, 'colsample_bytree': 0.6863629738189831, 'reg_alpha': 0.0016409881781172429, 'reg_lambda': 0.004709029793085598}
lgbm_model = lgbm_model.lgbm_model(lgbm_config)
lgbm_pipe = Pipeline([("scaler", StandardScaler()), ("lgbm", lgbm_model)])
lgbm_fitted = lgbm_pipe.fit(x_train, y_train)
lgbm_prediction = lgbm_fitted.predict(x_test)
lgbm_predicted_prices = np.expm1(lgbm_prediction)
lgbm_predicted_prices

In [ ]:
combined = (xgb_predicted_prices + lgbm_predicted_prices) / 2
test_labels = np.round([np.expm1(lab)for lab in y_test])

print("RMSLE for XGBoost model: %s" % xgboost_model.root_mean_squared_log_error(test_labels, xgb_predicted_prices))
print("RMSLE for LGBM model: %s\n" % xgboost_model.root_mean_squared_log_error(test_labels, lgbm_predicted_prices))

print("RMSLE for combined models: %s\n" % xgboost_model.root_mean_squared_log_error(test_labels, combined))